In this step of lab 1 you will code and understand basic steps of image classification, including cross-validation, using a KNN classifier.

Answers to the questions embedded in this code should be added to the file "submission.py".


In [43]:
#First, let's load in the data we created in the previous step.
import pickle

with open("testTrainLab1.pickle", "rb") as f:
    labData = pickle.load(f)

print("Confirm we have the expected 50,000 training Images:")
print(len(labData["X_train"]))
print(len(labData["y_train"]))

print("\nConfirm we have the expected 10,000 test Images:")
print(len(labData["X_test"]))
print(len(labData["y_test"]))

print("\nConfirm the first image in our training set is a 32x32x3 matrix of values, representing a 32x32 image with 3 bands of color:")
print(labData["X_train"][0].shape)
print(str(labData["X_train"][0].shape) == "(32, 32, 3)")

print("\nConfirm our y observations make sense - i.e., an integer between 0 and 9 representing one of the 10 classes in CIFAR")
print(labData["y_train"][0])
print(max(labData["y_train"]))
print(min(labData["y_train"]))

Confirm we have the expected 50,000 training Images:
50000
50000

Confirm we have the expected 10,000 test Images:
10000
10000

Confirm the first image in our training set is a 32x32x3 matrix of values, representing a 32x32 image with 3 bands of color:
(32, 32, 3)
True

Confirm our y observations make sense - i.e., an integer between 0 and 9 representing one of the 10 classes in CIFAR
6
9
0


In [44]:
import numpy as np
#Here, we're going to reshape our image data
#Right now, it's in matrices - i.e., the below will print out a matrix, 
#where each line of the matrix is a list:
print(labData["X_train"][0])

#While we could use that data, it's less effecient than doing array
#manipulations, where every value is in a single list.  Using array manipulations
#Greatly speeds up our code - while that isn't terribly important for this lab,
#it will be critical in the future, so let's get used to it now.
#This will reshape into rows - we'll also go ahead and unpack from our dict.
X_train = np.reshape(labData["X_train"], (labData["X_train"].shape[0], -1))
X_test = np.reshape(labData["X_test"], (labData["X_test"].shape[0], -1))

#Reshaped Matrix - note the first value is still '59', then '62', just like in the matrix form - it's just a long list now:"
print(X_train[0])

#Note our y values are just vectors, so nothing fancy we need to do.
#We could use the dictionary, but for consistency I am transforming these into
#variables.
y_train = labData["y_train"]
y_test = labData["y_test"]

[[[ 59.  62.  63.]
  [ 43.  46.  45.]
  [ 50.  48.  43.]
  ...
  [158. 132. 108.]
  [152. 125. 102.]
  [148. 124. 103.]]

 [[ 16.  20.  20.]
  [  0.   0.   0.]
  [ 18.   8.   0.]
  ...
  [123.  88.  55.]
  [119.  83.  50.]
  [122.  87.  57.]]

 [[ 25.  24.  21.]
  [ 16.   7.   0.]
  [ 49.  27.   8.]
  ...
  [118.  84.  50.]
  [120.  84.  50.]
  [109.  73.  42.]]

 ...

 [[208. 170.  96.]
  [201. 153.  34.]
  [198. 161.  26.]
  ...
  [160. 133.  70.]
  [ 56.  31.   7.]
  [ 53.  34.  20.]]

 [[180. 139.  96.]
  [173. 123.  42.]
  [186. 144.  30.]
  ...
  [184. 148.  94.]
  [ 97.  62.  34.]
  [ 83.  53.  34.]]

 [[177. 144. 116.]
  [168. 129.  94.]
  [179. 142.  87.]
  ...
  [216. 184. 140.]
  [151. 118.  84.]
  [123.  92.  72.]]]
[ 59.  62.  63. ... 123.  92.  72.]


In [47]:
#Here is our nearest neighbor classifier, 
#copied from the lecture notes.
#We're going to train it with our CIFAR data, then
#apply it (this will take a good while to run - as noted
#in lecture, this is a VERY ineffecient algorithm.)
#In the next snippet, we'll look at how well it does.

#Note this is a function that lets us build a loop
#That shows our progress in Jupyter, clearing the output
#and replacing it with the current iteration (out of 10,000)
from IPython.display import clear_output

class NearestNeighbor:
    def __init__(self):
        pass

    def train(self, X, y):
        self.Xtr = X
        self.ytr = y

    def predict(self, X):        
        Ypred = np.zeros(len(X), dtype=np.dtype(self.ytr.dtype))

        for i in range(0, len(X)):
            l1Distances = np.sum(np.abs(self.Xtr - X[i]), axis=1)
            minimumDistance = np.argmin(l1Distances)
            Ypred[i] = self.ytr[minimumDistance]
            
            #Added so we can see the code progress:
            clear_output(wait=True)
            print(str(i) + " (" + str((i/len(X))*100) + "%)")
        
        return Ypred

nnClass = NearestNeighbor()
nnClass.train(X_train, y_train)

yTestPredict = nnClass.predict(X_test)

249 (99.6%)


In [48]:
correctPred = np.sum(yTestPredict == y_test)
print("Our nearest neighbor classifier got it right " + str(correctPred) + " times out of our "+str(len(y_test))+" test images.")
print("That gives us an accuracy of " + str((correctPred / len(y_test)) * 100) + "%!")

Our nearest neighbor classifier got it right 88 times out of our 250 test images.
That gives us an accuracy of 35.199999999999996%!


In [46]:
#As you saw, nn is VERY slow.  Pretty obvious
#why it isn't used in practical cases!
#So, lets subset
#our testing data a bit to make things quicker.
#We'll just take the first few observations.
#If you re-run the above training and testing snippet
#It will nnow be much, much, much faster!

y_test = y_test[:250]
X_test = X_test[:250]

In [49]:
#Alright!  Now we want to extend our class to enable
#A choice of K - i.e., we want to choose
#A number of cases that are similar to what we want to predict
#Not just the single most similar case.

#You'll note in the below code that our training
#doesn't change at all - we're only modifying our prediction.

from IPython.display import clear_output

class KNearestNeighbor:
    def __init__(self):
        pass

    def train(self, X, y):
        self.Xtr = X
        self.ytr = y

    #First, in our prediction function we now need to take in a new
    #variable - k.
    def predict(self, X, k):        
        Ypred = np.zeros(len(X), dtype=np.dtype(self.ytr.dtype))

        for i in range(0, len(X)):
            l1Distances = np.sum(np.abs(self.Xtr - X[i]), axis=1)

            #In our older code, we just took the single
            #minimum distance, and then looked up the class
            #it belonged to.  Now we're going to do a vote
            #of the k closest.
            
            #First, we're going to look up the indices in our
            #array that represent the smallest values of distance.
            #It's sorted ascending, so the first index is the index
            #in the list l1Distances that is the smallest.
            #so, for example, l1Distances[minimumIndices[0]] would give
            #the smallest distance between the test image and all other images.
            minimumIndices = np.argsort(l1Distances)
            
            #Now, we want to grab the indices of the closest "k" matches:
            kClosest = minimumIndices[:k]
            
            #Now that we have the indices, we want to find the mode (i.e.,
            #the most common class of neighbors):
            #Note - ties are broken based on the order of the array, so for example
            #if a "1" is encountered before a "0", and 0 and 1 are tied, 1 "wins".
            predClass, counts = np.unique(self.ytr[kClosest], return_counts=True)
            Ypred[i] = predClass[counts.argmax()]
            
            #Added so we can see the code progress:
            clear_output(wait=True)
            print(str(i) + " (" + str((i/len(X))*100) + "%)")
        
        return Ypred

knnClass = KNearestNeighbor()
knnClass.train(X_train, y_train)

yTestPredictKNN = knnClass.predict(X_test, k=5)

correctPredKNN = np.sum(yTestPredictKNN == y_test)
print("Our nearest neighbor classifier got it right " + str(correctPredKNN) + " times out of our "+str(len(y_test))+" test images.")
print("That gives us an accuracy of " + str((correctPredKNN / len(y_test)) * 100) + "%!")

249 (99.6%)
Our nearest neighbor classifier got it right 90 times out of our 250 test images.
That gives us an accuracy of 36.0%!
